In [7]:
import pandas as pd
import os
import sys
sys.path.append("1_utils")
import functions as fun
import json

In [35]:
def check_price_alert(cardID, df, range_time="3m", sell_alert=0.75, buy_alert=0.25):
    today_vs_range, price_min_range, price_today, price_max_range = fun.price_analysis(df, cardID=cardID, range_time=range_time)
    if today_vs_range <= buy_alert:
        # BUY ALERT
        return (-1, cardID, range_time, today_vs_range, price_min_range, price_today, price_max_range)
    elif today_vs_range >= sell_alert:
        # SELL ALERT
        return (1, cardID, range_time, today_vs_range, price_min_range, price_today, price_max_range)
    else:
        # HOLD ALERT
        return (0, cardID, range_time, today_vs_range, price_min_range, price_today, price_max_range)



In [31]:
PATH_DATA_FOLDER = os.path.join("/home", "ubuntu", "lopezrbn", "goatbots_price_checker", "0_data")
PATH_DF_FINAL = os.path.join(PATH_DATA_FOLDER, "2_df_final", "df.parquet")
PATH_OTHER_DATA = os.path.join(PATH_DATA_FOLDER, "3_other_data")
PATH_FAV_CARDS = os.path.join(PATH_OTHER_DATA, "fav_cards.json")
PATH_SELL_ALERTS = os.path.join(PATH_OTHER_DATA, "sell_alerts.json")
PATH_BUY_ALERTS = os.path.join(PATH_OTHER_DATA, "buy_alerts.json")

In [8]:
with open(PATH_FAV_CARDS, "r") as f:
    fav_cards = json.load(f)
df = pd.read_parquet(PATH_DF_FINAL)

In [11]:
fav_cards

{'10043': {'name': 'Ancient Tomb', 'cardset': 'TE', 'foil': 0},
 '18245': {'name': 'Flooded Strand', 'cardset': 'ONS', 'foil': 0},
 '107243': {'name': 'Wrath of God', 'cardset': 'DMR', 'foil': 0}}

In [36]:
alerts = []
for cardID in fav_cards.keys():
    alerts.append(check_price_alert(cardID, df))
alerts

[(-1, '10043', '3m', 0.19, 18.74, 21.49, 33.49),
 (-1, '18245', '3m', 0.11, 7.19, 8.44, 18.99),
 (1, '107243', '3m', 1.0, 2.23, 2.82, 2.82)]

In [41]:
sell_alerts = {}
buy_alerts = {}
for alert in alerts:
    if alert[0] == 1:
        sell_alerts[alert[1]] = {
            "name": fav_cards[alert[1]]["name"],
            "cardset": fav_cards[alert[1]]["cardset"],
            "foil": fav_cards[alert[1]]["foil"],
            "range_time": alert[2],
            "today_vs_range": alert[3],
            "price_min_range": alert[4],
            "price_today": alert[5],
            "price_max_range": alert[6]
        }
    elif alert[0] == -1:
        buy_alerts[alert[1]] = {
            "name": fav_cards[alert[1]]["name"],
            "cardset": fav_cards[alert[1]]["cardset"],
            "foil": fav_cards[alert[1]]["foil"],
            "range_time": alert[2],
            "today_vs_range": alert[3],
            "price_min_range": alert[4],
            "price_today": alert[5],
            "price_max_range": alert[6]
        }

In [42]:
sell_alerts

{'107243': {'name': 'Wrath of God',
  'cardset': 'DMR',
  'foil': 0,
  'range_time': '3m',
  'today_vs_range': 1.0,
  'price_min_range': 2.23,
  'price_today': 2.82,
  'price_max_range': 2.82}}

In [43]:
buy_alerts

{'10043': {'name': 'Ancient Tomb',
  'cardset': 'TE',
  'foil': 0,
  'range_time': '3m',
  'today_vs_range': 0.19,
  'price_min_range': 18.74,
  'price_today': 21.49,
  'price_max_range': 33.49},
 '18245': {'name': 'Flooded Strand',
  'cardset': 'ONS',
  'foil': 0,
  'range_time': '3m',
  'today_vs_range': 0.11,
  'price_min_range': 7.19,
  'price_today': 8.44,
  'price_max_range': 18.99}}

In [44]:
with open(PATH_SELL_ALERTS, "w") as f:
    json.dump(sell_alerts, f)
with open(PATH_BUY_ALERTS, "w") as f:
    json.dump(buy_alerts, f)